# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb69cfcde80>
├── 'a' --> tensor([[-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607]])
└── 'x' --> <FastTreeValue 0x7fb69cfcde50>
    └── 'c' --> tensor([[ 1.4769, -1.3254,  0.0227, -0.1444],
                        [-0.4445,  0.4219,  0.5354, -1.3769],
                        [-0.0812, -0.4028, -0.6148, -2.4973]])

In [4]:
t.a

tensor([[-0.0416, -0.1654,  0.1618],
        [ 0.5258, -0.5184, -0.4607]])

In [5]:
%timeit t.a

78.9 ns ± 2.34 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb69cfcde80>
├── 'a' --> tensor([[-0.5918, -0.8060, -0.0314],
│                   [-0.7451, -1.2411,  0.0649]])
└── 'x' --> <FastTreeValue 0x7fb69cfcde50>
    └── 'c' --> tensor([[ 1.4769, -1.3254,  0.0227, -0.1444],
                        [-0.4445,  0.4219,  0.5354, -1.3769],
                        [-0.0812, -0.4028, -0.6148, -2.4973]])

In [7]:
%timeit t.a = new_value

85.4 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607]]),
    x: Batch(
           c: tensor([[ 1.4769, -1.3254,  0.0227, -0.1444],
                      [-0.4445,  0.4219,  0.5354, -1.3769],
                      [-0.0812, -0.4028, -0.6148, -2.4973]]),
       ),
)

In [10]:
b.a

tensor([[-0.0416, -0.1654,  0.1618],
        [ 0.5258, -0.5184, -0.4607]])

In [11]:
%timeit b.a

67.3 ns ± 0.731 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0575,  0.5402, -0.7430],
               [ 0.3695,  0.1340,  0.2362]]),
    x: Batch(
           c: tensor([[ 1.4769, -1.3254,  0.0227, -0.1444],
                      [-0.4445,  0.4219,  0.5354, -1.3769],
                      [-0.0812, -0.4028, -0.6148, -2.4973]]),
       ),
)

In [13]:
%timeit b.a = new_value

632 ns ± 7.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

297 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

301 µs ± 19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb69cfcde20>
├── 'a' --> tensor([[[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]],
│           
│                   [[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]],
│           
│                   [[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]],
│           
│                   [[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]],
│           
│                   [[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]],
│           
│                   [[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]],
│           
│                   [[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]],
│           
│                   [[-0.0416, -0.1654,  0.1618],
│                    [ 0.5258, -0.5184, -0.4607]]])
└── 'x' --> <FastTreeValue 0x7fb69cff6070>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.7 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb69cfcdbe0>
├── 'a' --> tensor([[-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607],
│                   [-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607],
│                   [-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607],
│                   [-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607],
│                   [-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607],
│                   [-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607],
│                   [-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607],
│                   [-0.0416, -0.1654,  0.1618],
│                   [ 0.5258, -0.5184, -0.4607]])
└── 'x' --> <FastTreeValue 0x7fb5f6da7490>
    └── 'c' --> tensor([[ 1.4769, -1.3254,  0.0227, -0.1444],
                        [-0.4445,  0.4219,  0.5354, -1.3769],
                 

In [23]:
%timeit t_cat(trees)

44.8 µs ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82.7 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]],
       
               [[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]],
       
               [[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]],
       
               [[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]],
       
               [[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]],
       
               [[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]],
       
               [[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]],
       
               [[-0.0416, -0.1654,  0.1618],
                [ 0.5258, -0.5184, -0.4607]]]),
    x: Batch(
           c: tensor([[[ 1.4769, -1.3254,  0.0227, -0.1444],
                       [-0.4445,  0.4219,  0.5354, -1.3769],
                       [-0.0812, -0.4028, -0.6148, -2.4973]],
         

In [26]:
%timeit Batch.stack(batches)

106 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607],
               [-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607],
               [-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607],
               [-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607],
               [-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607],
               [-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607],
               [-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607],
               [-0.0416, -0.1654,  0.1618],
               [ 0.5258, -0.5184, -0.4607]]),
    x: Batch(
           c: tensor([[ 1.4769, -1.3254,  0.0227, -0.1444],
                      [-0.4445,  0.4219,  0.5354, -1.3769],
                      [-0.0812, -0.4028, -0.6148, -2.4973],
                      [ 1.4769, -1.3254,  0.0227, -0.1444],
                      [-0.4445,  

In [28]:
%timeit Batch.cat(batches)

196 µs ± 5.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

616 µs ± 8.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
